<a href="https://colab.research.google.com/github/xvinh74/data-science/blob/main/TextGNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/congnghia0609/ntc-scv.git
!unzip ./ntc-scv/data/data_test.zip -d ./data
!unzip ./ntc-scv/data/data_train.zip -d ./data
!rm -rf ./ntc-scv

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: ./data/data_train/train/pos/31228.txt  
  inflating: ./data/data_train/train/pos/3123.txt  
  inflating: ./data/data_train/train/pos/31232.txt  
  inflating: ./data/data_train/train/pos/31233.txt  
  inflating: ./data/data_train/train/pos/31234.txt  
  inflating: ./data/data_train/train/pos/31236.txt  
  inflating: ./data/data_train/train/pos/31238.txt  
  inflating: ./data/data_train/train/pos/31240.txt  
  inflating: ./data/data_train/train/pos/31242.txt  
  inflating: ./data/data_train/train/pos/31247.txt  
  inflating: ./data/data_train/train/pos/31250.txt  
  inflating: ./data/data_train/train/pos/31251.txt  
  inflating: ./data/data_train/train/pos/31252.txt  
  inflating: ./data/data_train/train/pos/31257.txt  
  inflating: ./data/data_train/train/pos/31260.txt  
  inflating: ./data/data_train/train/pos/31261.txt  
  inflating: ./data/data_train/train/pos/31262.txt  
  inflating: ./data/data_train/train/pos/31

In [2]:
# load data from paths to dataframe
import os
import pandas as pd

def load_data_from_path(folder_path):
	examples = []
	for label in os.listdir(folder_path):
		full_path = os.path.join(folder_path, label)
		for file_name in os.listdir(full_path):
			file_path = os.path.join(full_path, file_name)
			with open (file_path , "r" , encoding="utf-8") as f:
				lines = f.readlines()
			sentence = " ".join(lines)
			if label == "neg":
				label = 0
			if label == "pos":
				label = 1
			data = {
				'sentence': sentence,
				'label': label
			}
			examples.append(data)
	return pd.DataFrame(examples)

In [3]:
folder_paths = {
	'train': '/content/data/data_train/train',
	'valid': '/content/data/data_train/test',
	'test': '/content/data/data_train/test'
}

In [4]:
train_df = load_data_from_path(folder_paths['train'])
valid_df = load_data_from_path(folder_paths['valid'])
test_df = load_data_from_path(folder_paths['test'])

In [5]:
# Install library
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=6b98472e0ca258496caa336c38f4ff170df55ab29bad225d26e1b15dadd2a92a
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [6]:
from langid.langid import LanguageIdentifier, model

def identify_vn(df):
	identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
	not_vi_idx = set()
	THRESHOLD = 0.9
	for idx, row in df.iterrows():
		score = identifier.classify(row["sentence"])
		if score[0] != "vi" or (score[0] == "vi" and score[1]<= THRESHOLD):
			not_vi_idx.add(idx)
	vi_df = df[~df.index.isin(not_vi_idx)]
	not_vi_df = df[df.index.isin(not_vi_idx)]
	return vi_df, not_vi_df

train_df_vi, train_df_other = identify_vn(train_df)

In [7]:
import re
import string

def preprocess_text(text):
    # Loại bỏ URL
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(" ", text)

    # Loại bỏ thẻ HTML
    html_pattern = re.compile(r'<.*?>')
    text = html_pattern.sub(" ", text)

    # Loại bỏ ký tự đặc biệt và số
    text = text.translate(str.maketrans("", "", string.punctuation + string.digits))

    # Loại bỏ emoji
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"  # other symbols
        u"\U000024C2-\U0001F251"  # enclosed characters
        u"\u200d"                 # zero width joiner
        u"\u2640-\u2642"          # gender symbols
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(" ", text)

    # Xóa khoảng trắng thừa và chuyển thành chữ thường
    text = " ".join(text.split())
    return text.lower()


In [8]:
train_df_vi['preprocess_sentence'] = [
	preprocess_text(row['sentence']) for index, row in train_df_vi.iterrows()
]

valid_df['preprocess_sentence'] = [
	preprocess_text(row['sentence'])for index, row in valid_df.iterrows()
]

test_df['preprocess_sentence'] = [
	preprocess_text(row['sentence']) for index, row in test_df.iterrows()
]

<ipython-input-8-c8f49442c03b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_vi['preprocess_sentence'] = [


In [12]:
!pip install -q torchtext==0.16.0

In [13]:
# word-based tokenizer
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer("basic_english")

# create iter dataset
def yield_tokens(sentences, tokenizer):
	for sentence in sentences:
		yield tokenizer(sentence)

# build vocabulary
from torchtext.vocab import build_vocab_from_iterator

vocab_size = 10000
vocabulary = build_vocab_from_iterator(
	yield_tokens(train_df_vi['preprocess_sentence'], tokenizer),
	max_tokens = vocab_size,
	specials = ["<pad>", "<unk>"]
)
vocabulary.set_default_index(vocabulary["<unk>"])

# convert iter into torchtext dataset
from torchtext.data.functional import to_map_style_dataset

def prepare_dataset(df):
	for index, row in df.iterrows():
		sentence = row['preprocess_sentence']
		encoded_sentence = vocabulary(tokenizer(sentence))
		label = row['label']
		yield encoded_sentence, label

train_dataset = prepare_dataset(train_df_vi)
train_dataset = to_map_style_dataset(train_dataset)

valid_dataset = prepare_dataset(valid_df)
valid_dataset = to_map_style_dataset(valid_dataset)

In [19]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
	# create inputs, offsets, labels for batch
	encoded_sentences, labels = [] , []

	for encoded_sentence, label in batch:
		labels.append(label)
		encoded_sentence = torch.tensor(encoded_sentence, dtype=torch.int64)
		encoded_sentences.append(encoded_sentence)

	labels = torch.tensor(labels, dtype=torch.int64)
	encoded_sentences = pad_sequence (
		encoded_sentences,
		padding_value = vocabulary["<pad>"]
	)

	return encoded_sentences , labels

batch_size = 128
train_dataloader = DataLoader (
	train_dataset,
	batch_size=batch_size,
	shuffle=True,
	collate_fn=collate_batch
)

valid_dataloader = DataLoader(
	valid_dataset,
	batch_size=batch_size,
	shuffle=False,
	collate_fn=collate_batch
)

In [20]:
# import torch
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, kernel_sizes, num_filters, num_classes):
        super(TextCNN, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.kernel_sizes = kernel_sizes
        self.num_filters = num_filters
        self.num_classes = num_classes

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # Convolutional layers
        self.conv = nn.ModuleList([
            nn.Conv1d(
                in_channels=embedding_dim,
                out_channels=num_filters,
                kernel_size=k,
                stride=1
            ) for k in kernel_sizes
        ])

        # Fully connected layer
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)

    def forward(self, x):
        # Input shape: (batch_size, sequence_length)
        batch_size, sequence_length = x.shape

        # Embedding layer output: (batch_size, sequence_length, embedding_dim)
        x = self.embedding(x)

        # Transpose for Conv1d: (batch_size, embedding_dim, sequence_length)
        x = x.transpose(1, 2)

        # Apply convolutional layers + ReLU
        x = [F.relu(conv(x)) for conv in self.conv]

        # Apply max pooling: (batch_size, num_filters, 1) -> squeeze to (batch_size, num_filters)
        x = [F.max_pool1d(c, c.size(-1)).squeeze(dim=-1) for c in x]

        # Concatenate along feature dimension: (batch_size, len(kernel_sizes) * num_filters)
        x = torch.cat(x, dim=1)

        # Fully connected layer
        x = self.fc(x)

        return x


In [23]:
# Training
import time
#import torch

def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    total_loss = 0  # Tổng loss
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero grad
        optimizer.zero_grad()

        # Predictions
        predictions = model(inputs)

        # Compute loss
        loss = criterion(predictions, labels)
        total_loss += loss.item()

        # Backward
        loss.backward()
        optimizer.step()

        # Accuracy
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)

        # Log every log_interval batches
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                f"| epoch {epoch:3d} | batch {idx:5d}/{len(train_dataloader):5d} "
                f"| accuracy {total_acc / total_count:8.3f} | elapsed time {elapsed:.2f}s"
            )

    # Accuracy and loss for the epoch
    epoch_acc = total_acc / total_count
    epoch_loss = total_loss / len(train_dataloader)

    print(f"Epoch {epoch} completed: Accuracy = {epoch_acc:.3f}, Loss = {epoch_loss:.3f}")
    return epoch_acc, epoch_loss

# Evaluation
def evaluate(model, criterion, valid_dataloader, device):
    model.eval()
    total_acc, total_count = 0, 0
    total_loss = 0  # Tổng loss

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Predictions
            predictions = model(inputs)

            # Compute loss
            loss = criterion(predictions, labels)
            total_loss += loss.item()

            # Accuracy
            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    # Accuracy and loss for the evaluation
    epoch_acc = total_acc / total_count
    epoch_loss = total_loss / len(valid_dataloader)

    print(f"Validation: Accuracy = {epoch_acc:.3f}, Loss = {epoch_loss:.3f}")
    return epoch_acc, epoch_loss
